In [1]:
import os
import glob
import h5py
import numpy as np
from keras import backend as K
from keras.models import Sequential, load_model
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten, Dense, Activation, Reshape
from keras.layers import Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.callbacks import Callback

Using TensorFlow backend.


In [2]:
print(K.image_dim_ordering())
img_width, img_height = 150, 150
root_train_folder_path = os.path.join('database', 'train')
train_data_dir = os.path.join(root_train_folder_path, 'train')
validation_data_dir = os.path.join(root_train_folder_path, 'validation')

tf


In [3]:
# used to rescale the pixel values from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1./255)

In [4]:
# automagically retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=16,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=4,
        class_mode='binary')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [5]:
nb_train_samples = 20000
nb_validation_samples = 500

In [6]:
models_folder_path = os.path.join("models")
model_name = "simple_with_dropout"
model = load_model(os.path.join(models_folder_path, model_name) + '-steps{}.h5'.format(60))

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1_1 (Conv2D)             (None, 150, 150, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 150, 150, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 64)        0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 75, 75, 128)       73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 75, 75, 128)       147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 37, 37, 128)       0         
_________________________________________________________________
conv3_1 (Conv2D)             (None, 37, 37, 256)       295168    
__________

In [ ]:
class ModelCheckpoint(Callback):
    def __init__(self, model_ptr, model_path):
        self.curr_model = model_ptr
        self.model_path = model_path
    
    def on_train_begin(self, logs={}):
        # print("train start!!")
        pass
    
    def on_batch_end(self, batch, logs={}):
        # print("batch start!! ", batch)
        pass
    
    def on_epoch_end(self, epoch, logs={}):
        self.curr_model.save(self.model_path + '-epoch{}.h5'.format(epoch))

In [ ]:
models_folder_path = os.path.join('database', 'models')
model_name = "continue_callback_model"
model_name_path = os.path.join(models_folder_path, model_name)

checkpoints = ModelCheckpoint(model, model_name_path)

model.fit_generator(
        train_generator,
        steps_per_epoch=2000,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=800,
        verbose=1,
        callbacks=[checkpoints])

Epoch 1/50
2000/2000 [==============================] - 437s - loss: 0.1722 - acc: 0.9305 - val_loss: 0.2346 - val_acc: 0.9066
Epoch 2/50
2000/2000 [==============================] - 364s - loss: 0.1669 - acc: 0.9324 - val_loss: 0.2611 - val_acc: 0.8912
Epoch 3/50
2000/2000 [==============================] - 351s - loss: 0.1586 - acc: 0.9379 - val_loss: 0.2427 - val_acc: 0.9050
Epoch 4/50
2000/2000 [==============================] - 351s - loss: 0.1511 - acc: 0.9394 - val_loss: 0.2390 - val_acc: 0.9050
Epoch 5/50
2000/2000 [==============================] - 351s - loss: 0.1475 - acc: 0.9417 - val_loss: 0.2700 - val_acc: 0.8959
Epoch 6/50
2000/2000 [==============================] - 351s - loss: 0.1411 - acc: 0.9427 - val_loss: 0.3132 - val_acc: 0.8866
Epoch 7/50
2000/2000 [==============================] - 351s - loss: 0.1311 - acc: 0.9479 - val_loss: 0.2300 - val_acc: 0.9178
Epoch 8/50
2000/2000 [==============================] - 351s - loss: 0.1268 - acc: 0.9486 - val_loss: 0.2520 - 